In [1]:
import pickle

In [8]:
# open a file, where you stored the pickled data
file = open('wine_word2vec_model', 'rb')

# dump information to that file
wine_word2vec_model = pickle.load(file)

# close the file
file.close()

# Same for the rest of the files
file = open('model_knn', 'rb')
model_knn = pickle.load(file)
file.close()

file = open('dict_of_tfidf_weightings', 'rb')
dict_of_tfidf_weightings = pickle.load(file)
file.close()

file = open('wine_reviews_mincount', 'rb')
wine_reviews_mincount = pickle.load(file)
file.close()

In [9]:
def descriptors_to_best_match_wines(list_of_descriptors, number_of_suggestions=10):
    weighted_review_terms = []
    for term in list_of_descriptors:
        if term not in dict_of_tfidf_weightings:
            if term not in descriptor_mapping.index:
                print('choose a different descriptor from', term)
                continue
            else:
                term = descriptor_mapping['normalized'][term]
        tfidf_weighting = dict_of_tfidf_weightings[term]
        word_vector = wine_word2vec_model.wv.get_vector(term).reshape(1, 300)
        weighted_word_vector = tfidf_weighting * word_vector
        weighted_review_terms.append(weighted_word_vector)
    review_vector = sum(weighted_review_terms)

    distance, indice = model_knn.kneighbors(review_vector, n_neighbors=number_of_suggestions+1)
    distance_list = distance[0].tolist()[1:]
    indice_list = indice[0].tolist()[1:]

    n = 1
    for d, i in zip(distance_list, indice_list):
        wine_name = wine_reviews_mincount['title'][i]
        wine_descriptors = wine_reviews_mincount['normalized_descriptors'][i]
        print('Suggestion', str(n), ':', wine_name, 'with a cosine distance of', "{:.3f}".format(d))
        print('This wine has the following descriptors:', wine_descriptors)
        print('')
        n+=1

In [10]:
descriptors = ['apple','pear','wood']
descriptors_to_best_match_wines(list_of_descriptors=descriptors, number_of_suggestions=5)

Suggestion 1 : Domaine Faiveley 2011 Les Villeranges  (Rully) with a cosine distance of 0.084
This wine has the following descriptors: ['fresh', 'fruit', 'citrus', 'pear', 'apple', 'wood', 'round']

Suggestion 2 : Domaine Ostertag 2010 Barriques Pinot Blanc (Alsace) with a cosine distance of 0.088
This wine has the following descriptors: ['oak', 'wood', 'toast', 'pear', 'honey', 'apple']

Suggestion 3 : A.R. Lenoble  NV Intense Brut  (Champagne) with a cosine distance of 0.110
This wine has the following descriptors: ['wood', 'rich', 'ripe', 'fruit', 'pear', 'crisp', 'apple', 'apricot']

Suggestion 4 : Domaine de la Garenne 2008  Mâcon-Azé with a cosine distance of 0.120
This wine has the following descriptors: ['wood', 'fruit', 'fresh', 'grapefruit', 'pear', 'bright', 'lemon', 'honey', 'wood']

Suggestion 5 : Wheelhouse 2011 Chardonnay (Sonoma County) with a cosine distance of 0.126
This wine has the following descriptors: ['ripe', 'apple', 'pear', 'tropical_fruit', 'fruit', 'oak']

